In [1]:
import tensorflow as tf
import numpy as np
tf.InteractiveSession()

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x1 = tf.eye(5,batch_shape=[3])
x2 = tf.random_uniform([5,2,3])
x = tf.convert_to_tensor(np.array([[[1,2,3,4],[10,11,12,13]],[[1,2,3,4],[10,11,12,13]],[[3,2,4,5],[8,13,2,1]]]), dtype='float32')

In [3]:
#x = tf.expand_dims(x,2)
x.eval()

array([[[ 1.,  2.,  3.,  4.],
        [10., 11., 12., 13.]],

       [[ 1.,  2.,  3.,  4.],
        [10., 11., 12., 13.]],

       [[ 3.,  2.,  4.,  5.],
        [ 8., 13.,  2.,  1.]]], dtype=float32)

In [4]:
concat_within = lambda row: tf.concat([row,x[0,:]],1)
#y = tf.concat([x[0,],x[0,:]],1)


In [5]:
els = tf.map_fn(concat_within, x, dtype=tf.float32)[1:,]

In [6]:
els.eval()

array([[[ 1.,  2.,  3.,  4.,  1.,  2.,  3.,  4.],
        [10., 11., 12., 13., 10., 11., 12., 13.]],

       [[ 3.,  2.,  4.,  5.,  1.,  2.,  3.,  4.],
        [ 8., 13.,  2.,  1., 10., 11., 12., 13.]]], dtype=float32)

In [7]:
els = tf.reshape(els,[els.shape[0]*els.shape[1],els.shape[2]])

In [8]:
els.eval()

array([[ 1.,  2.,  3.,  4.,  1.,  2.,  3.,  4.],
       [10., 11., 12., 13., 10., 11., 12., 13.],
       [ 3.,  2.,  4.,  5.,  1.,  2.,  3.,  4.],
       [ 8., 13.,  2.,  1., 10., 11., 12., 13.]], dtype=float32)

In [12]:
encoder_embeds = 4
condense_embeds = 2
def condense(inp):
    with tf.variable_scope("condense"):
        W = tf.get_variable("W", dtype = tf.float32, shape=[2*encoder_embeds,condense_embeds])
        b=  tf.get_variable("b", dtype = tf.float32, shape=[condense_embeds], initializer= tf.zeros_initializer())
        return tf.matmul(inp,W) + b


In [13]:
out = condense(els)

In [14]:
tf.global_variables_initializer().run() # or 
tf.initialize_all_variables().run()
out.eval()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


array([[ -2.202735 ,   5.1193056],
       [-13.070811 ,  19.441303 ],
       [ -2.9686675,   6.2830596],
       [ -3.753151 ,  14.768406 ]], dtype=float32)

In [15]:
out.shape

TensorShape([Dimension(4), Dimension(2)])

In [17]:
out = tf.reshape(out, [-1,2,2])

In [18]:
out.eval()

array([[[ -2.202735 ,   5.1193056],
        [-13.070811 ,  19.441303 ]],

       [[ -2.9686675,   6.2830596],
        [ -3.753151 ,  14.768406 ]]], dtype=float32)

In [11]:
xh = x[:,:,:2]
xc = x[:,:,2:]

In [12]:
xh.eval()

array([[[ 1.,  2.],
        [10., 11.]],

       [[ 1.,  2.],
        [10., 11.]],

       [[ 3.,  2.],
        [ 8., 13.]]], dtype=float32)

In [30]:
normalized_xh = tf.nn.l2_normalize(xh, dim = 2)
normalized_xc = tf.nn.l2_normalize(xc, dim = 2)


In [31]:
mul_h = tf.multiply(normalized_xh, normalized_xh[0,:,])[1:]
mul_c = tf.multiply(normalized_xc, normalized_xc[0,:,])[1:]

In [33]:
cos_similar_h = tf.reduce_sum(mul_h, 2, keep_dims=True)
cos_similar_c = tf.reduce_sum(mul_c, 2, keep_dims=True)
cos_similar_h.eval()

array([[[0.99999994],
        [0.9999999 ]],

       [[0.8682431 ],
        [0.98272216]]], dtype=float32)

In [34]:
cos_similar = tf.concat([cos_similar_h,cos_similar_c],axis =2)

In [35]:
cos_similar.eval()

array([[[0.99999994, 1.        ],
        [0.9999999 , 0.9999997 ]],

       [[0.8682431 , 0.9995121 ],
        [0.98272216, 0.93528605]]], dtype=float32)

In [97]:
cos_similar_batch_major = tf.transpose(cos_similar, perm = [1,0,2])

In [99]:
cos_similar_batch_major.

array([[[0.9999999 ],
        [0.94295406]],

       [[0.9999999 ],
        [0.83980083]]], dtype=float32)

In [52]:
normalize.shape

TensorShape([Dimension(3), Dimension(2), Dimension(3)])

In [39]:
mul = lambda x: tf.multiply(x,tf.expand_dims(x1[0,:,],1))
#y_cos = tf.map_fn(tf.map_fn(mul, x1),x1)

TypeError: fn must be callable.

In [34]:
y_cos.eval()

array([[[0.9999999, 0.       , 0.       , 0.       , 0.       ],
        [0.       , 0.9999999, 0.       , 0.       , 0.       ],
        [0.       , 0.       , 0.9999999, 0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.9999999, 0.       ],
        [0.       , 0.       , 0.       , 0.       , 0.9999999]],

       [[0.9999999, 0.       , 0.       , 0.       , 0.       ],
        [0.       , 0.9999999, 0.       , 0.       , 0.       ],
        [0.       , 0.       , 0.9999999, 0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.9999999, 0.       ],
        [0.       , 0.       , 0.       , 0.       , 0.9999999]],

       [[0.9999999, 0.       , 0.       , 0.       , 0.       ],
        [0.       , 0.9999999, 0.       , 0.       , 0.       ],
        [0.       , 0.       , 1.       , 0.       , 0.       ],
        [0.       , 0.       , 0.       , 1.       , 0.       ],
        [0.       , 0.       , 0.       , 0.       , 1.       ]]],
      dtype=float32

In [19]:
x1.eval()

array([[[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]], dtype=float32)

In [46]:
#Given a tensor like [[1,2,3,0,0,0],[3,4,5,6,0,0]]
#convert to [[[1,2,3,0,0,0],[2,3,0,0,0],[1,3,0,0,0],[1,2,0,0,0],etc]]

#We first output for lengths of n again (where n is max length)
#Then we concat to original column


#IN seq2seq processing, make sure to read in the seq_length.<- Very imp

In [47]:
#We plan to apply a map_fn to each which does that
#The map_fn creates a new batch by applying ignore given indices

In [10]:
import tensorflow as tf
import numpy as np 
import time

In [49]:
shape_input_word_seq = tf.shape(input_word_seq_tensor)

In [31]:
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)

In [ ]:
re = tf

In [8]:
#resultant_tensor.get_shape()
tf.InteractiveSession()

In [29]:
tf.TensorShape(shape_input_word_seq)

TypeError: int() argument must be a string or a number, not 'Tensor'

In [27]:
tf.TensorShape([None,5,5])

TensorShape([Dimension(None), Dimension(5), Dimension(5)])

In [19]:

tf.ones(shape=(seq_lengths.shape[0],),dtype="int32"), tf.convert_to_tensor(np.ones(shape=(seq_lengths.shape[0]),dtype="int32"))

(<tf.Tensor 'ones_13:0' shape=(2,) dtype=int32>,
 <tf.Tensor 'Const_60:0' shape=(2,) dtype=int32>)

In [13]:
tf.InteractiveSession()

input_word_seq_tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[1,2,3,4,3,2]]),dtype='int32')
max_seq_length = 6
seq_lengths = tf.convert_to_tensor(np.array([4,5]),dtype='int32') #shape n
'''Create mask based on max seq length'''

t1 = time.time()
np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
a = np.array(range(max_seq_length))
np_mask_matrix[np.arange(len(a)), a] = 0
#np_mask_matrix  = np_mask_matrix!=0
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
padding = tf.constant([[0,0],[0,1]],dtype='int32')
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)
#print(resultant_tensor.eval())
tensor_seq_lengths = tf.expand_dims(seq_lengths, 0)
seq_lengths_for_dropped = tf.expand_dims(seq_lengths-tf.ones(shape=(seq_lengths.shape[0],),dtype="int32"),0)

drop_index = tf.constant(0)
max_sequence_length = tf.convert_to_tensor(max_seq_length, dtype="int32")
def condition(resultant_tensor, tensor_seq_lengths,drop_index):
    return drop_index< max_seq_length

def body(resultant_tensor, tensor_seq_lengths,drop_index):
    f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
    '''Apply masking and padding'''
    temp = tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 
    temp = tf.expand_dims(temp,0)
    resultant_tensor = tf.concat([resultant_tensor,temp],0)
    tensor_seq_lengths = tf.concat([tensor_seq_lengths, seq_lengths_for_dropped],0)
    drop_index+=1
    return resultant_tensor, tensor_seq_lengths, drop_index

result = resultant_tensor, tensor_seq_lengths, drop_index = tf.while_loop(condition, 
                                                                 body, 
                                                                 [resultant_tensor, tensor_seq_lengths,0], 
                                                                 shape_invariants= [tf.TensorShape([None,input_word_seq_tensor.shape[0],max_seq_length]),tf.TensorShape([None,input_word_seq_tensor.shape[0],]),drop_index.get_shape()])


resultant_tensor_shape = tf.shape(resultant_tensor)
#tensor_seq_shape = tf.shape()
resultant_tensor = tf.reshape(resultant_tensor,[resultant_tensor_shape[0]*resultant_tensor_shape[1],resultant_tensor_shape[2]])
tensor_seq_lengths = tf.reshape(tensor_seq_lengths,[resultant_tensor_shape[0]*resultant_tensor_shape[1],])
x= resultant_tensor.eval()

print(resultant_tensor.eval(), tensor_seq_lengths.eval(), drop_index.eval())
t2 = time.time()
print(t2-t1)
#resultant_tensor, tensor_seq_lengths, drop_index = ([resultant_tensor, tensor_seq_lengths, drop_index])
#seq2seq_in = tf.reshape(resultant_tensor,[resultant_tensor.shape[0]*resultant_tensor.shape[1],resultant_tensor.shape[2]])
#seq2seq_in_lengths = tf.reshape(tensor_seq_lengths,[tensor_seq_lengths.shape[0]*tensor_seq_lengths.shape[1],])

#print(seq2seq_in.eval())
#print(seq2seq_in.shape)
#print(seq2seq_in_lengths.eval())
#print(seq2seq_in_lengths.shape)

(array([[0, 1, 2, 3, 1, 2],
       [1, 2, 3, 4, 3, 2],
       [1, 2, 3, 1, 2, 0],
       [2, 3, 4, 3, 2, 0],
       [0, 2, 3, 1, 2, 0],
       [1, 3, 4, 3, 2, 0],
       [0, 1, 3, 1, 2, 0],
       [1, 2, 4, 3, 2, 0],
       [0, 1, 2, 1, 2, 0],
       [1, 2, 3, 3, 2, 0],
       [0, 1, 2, 3, 2, 0],
       [1, 2, 3, 4, 2, 0],
       [0, 1, 2, 3, 1, 0],
       [1, 2, 3, 4, 3, 0]], dtype=int32), array([4, 5, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4], dtype=int32), 6)
0.900222063065


In [9]:
x

array([[[0, 1, 2, 3, 1, 2],
        [1, 2, 3, 4, 3, 2]],

       [[1, 2, 3, 1, 2, 0],
        [2, 3, 4, 3, 2, 0]],

       [[0, 2, 3, 1, 2, 0],
        [1, 3, 4, 3, 2, 0]],

       [[0, 1, 3, 1, 2, 0],
        [1, 2, 4, 3, 2, 0]],

       [[0, 1, 2, 1, 2, 0],
        [1, 2, 3, 3, 2, 0]],

       [[0, 1, 2, 3, 2, 0],
        [1, 2, 3, 4, 2, 0]],

       [[0, 1, 2, 3, 1, 0],
        [1, 2, 3, 4, 3, 0]]], dtype=int32)

In [69]:
import pickle

In [70]:
with open("../data/vocab_to_id.pkl",'r') as p1:
    x = pickle.load(p1)
x_inv = {val:key for key,val in x.items()}

In [76]:
'.' == 'empty .'[-1]

True

In [80]:
x['i.e']
#x_inv[4327]

KeyError: 'i.e'

In [38]:
'''FINAL SCRIPT'''

'''To do:
1) Integrate with model and within session
1) Move stuff that can be moved outside loops
2) Try not adding more nodes to the graph
'''
def get_encoder_reps(resultant_tensor, tensor_seq_lengths):
    return tf.convert_to_tensor(np.random.random_sample((resultant_tensor.shape[0],50)))


tf.InteractiveSession()

'''INPUTS'''
input_word_seq_tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[9,10,11,12,13,12]]),dtype='int32')
max_seq_length = 6
enc_dims = 50
seq_lengths = tf.convert_to_tensor(np.array([4,5]),dtype='int32') #shape n



t1 = time.time()
'''Create mask based on max seq length'''
np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
a = np.array(range(max_seq_length))
np_mask_matrix[np.arange(len(a)), a] = 0
#np_mask_matrix  = np_mask_matrix!=0
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
padding = tf.constant([[0,0],[0,1]],dtype='int32')
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)
#print(resultant_tensor.eval())
tensor_seq_lengths = tf.expand_dims(seq_lengths, 0)
seq_lengths_for_dropped = tf.expand_dims(seq_lengths-tf.ones(shape=seq_lengths.shape[0],dtype="int32"),0)
#print(seq_lengths_for_dropped)

#def condition(x):
 #   x.shape[0]< max_seq_length
#def body(x):
 #   Copy

for drop_index in range(max_seq_length):
    '''Create mask function-> note the drop index changes at each iteration'''
    f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
    '''Apply masking and padding'''
    #temp = tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 
    #temp = tf.expand_dims(temp,0)
    resultant_tensor = tf.concat([resultant_tensor,tf.expand_dims(tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT"),0)],0)
    tensor_seq_lengths = tf.concat([tensor_seq_lengths, seq_lengths_for_dropped],0)
#print(resultant_tensor.eval())
#print(resultant_tensor.shape)
#print(tensor_seq_lengths.eval())
#print(tensor_seq_lengths.shape)
resultant_tensor = tf.reshape(resultant_tensor,[resultant_tensor.shape[0]*resultant_tensor.shape[1],resultant_tensor.shape[2]])
tensor_seq_lengths = tf.reshape(tensor_seq_lengths,[tensor_seq_lengths.shape[0]*tensor_seq_lengths.shape[1],])
#seq2seq_out = get_encoded_rep()
#the seq2seq_out = reshaping of
'''get_encoder_reps returns the batched encoder representations (shape = [(input_max_length+2)*(input_batch_size) , encoder_dims]'''
x = get_encoder_reps(resultant_tensor,tensor_seq_lengths)
seq2seq_out = tf.reshape(x, [max_seq_length+1, input_word_seq_tensor.shape[0], enc_dims])

#Each column corresponds to the vector rep for a sentence
#Each row corresponds to the word representation

#We now reduce this by doing an op(row_0,row_i) for all columns
'''NOTE : Hard coded here with subtract have to replace with op(row_0, row_i) for all rows i'''
seq2seq_out = tf.subtract(seq2seq_out,seq2seq_out[0,:])[1:,:]
#So final output should be max_seq_len*n_batches*dims --> do transpose if needed
#seq2seq_out = tf.transpose(seq2seq_out,perm=[1,0,2]) #convert to n_batches*max_seq_len*dims or n_batches*words*dims




#apply function operation that takes first index and performs op(first_index, slice)

print(resultant_tensor.eval())
print(tensor_seq_lengths.eval())
#print(seq2seq_in.shape)
#print(seq2seq_in_lengths.eval())
#print(seq2seq_in_lengths.shape)
print(x.eval())
print(seq2seq_out.eval())

t2 = time.time()
print(t2-t1)

[[ 0  1  2  3  1  2]
 [ 9 10 11 12 13 12]
 [ 1  2  3  1  2  0]
 [10 11 12 13 12  0]
 [ 0  2  3  1  2  0]
 [ 9 11 12 13 12  0]
 [ 0  1  3  1  2  0]
 [ 9 10 12 13 12  0]
 [ 0  1  2  1  2  0]
 [ 9 10 11 13 12  0]
 [ 0  1  2  3  2  0]
 [ 9 10 11 12 12  0]
 [ 0  1  2  3  1  0]
 [ 9 10 11 12 13  0]]
[4 5 3 4 3 4 3 4 3 4 3 4 3 4]
[[0.37692606 0.0892903  0.05837255 0.93667809 0.5931503  0.01183983
  0.87316713 0.67405908 0.77440199 0.176826   0.08036772 0.00909012
  0.35495671 0.55432181 0.99279771 0.1692256  0.03690108 0.08341613
  0.21411401 0.44366764 0.152348   0.58297719 0.49090473 0.31181382
  0.23577738 0.13224074 0.62074842 0.54795951 0.07432954 0.78303944
  0.48774723 0.18749248 0.36618764 0.02638166 0.82247658 0.15944064
  0.01777281 0.53140054 0.66061428 0.93751091 0.31606636 0.56341985
  0.14131054 0.6372047  0.05883968 0.63111267 0.38493256 0.33363723
  0.44941909 0.33834288]
 [0.78287971 0.9023123  0.76554064 0.24096218 0.91207525 0.49829968
  0.05322931 0.49511398 0.9641632  0.5

In [3]:

'''INPUTS'''
input_word_seq_tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[9,10,11,12,13,12]]),dtype='int32')
max_seq_length = 6
enc_dims = 50
seq_lengths = tf.convert_to_tensor(np.array([4,5]),dtype='int32') #shape n




'''Create mask based on max seq length'''
np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
a = np.array(range(max_seq_length))
np_mask_matrix[np.arange(len(a)), a] = 0
#np_mask_matrix  = np_mask_matrix!=0
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
padding = tf.constant([[0,0],[0,1]],dtype='int32')
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)
#print(resultant_tensor.eval())
tensor_seq_lengths = tf.expand_dims(seq_lengths, 0)
seq_lengths_for_dropped = tf.expand_dims(seq_lengths-tf.ones(shape=seq_lengths.shape[0],dtype="int32"),0)
#print(seq_lengths_for_dropped)

#def condition(x):
 #   x.shape[0]< max_seq_length
#def body(x):
 #   Copy

for drop_index in range(max_seq_length):
    '''Create mask function-> note the drop index changes at each iteration'''
    f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
    '''Apply masking and padding'''
    #temp = tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 
    #temp = tf.expand_dims(temp,0)
    resultant_tensor = tf.concat([resultant_tensor,tf.expand_dims(tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT"),0)],0)
    tensor_seq_lengths = tf.concat([tensor_seq_lengths, seq_lengths_for_dropped],0)
#print(resultant_tensor.eval())
#print(resultant_tensor.shape)
#print(tensor_seq_lengths.eval())
#print(tensor_seq_lengths.shape)
resultant_tensor = tf.reshape(resultant_tensor,[resultant_tensor.shape[0]*resultant_tensor.shape[1],resultant_tensor.shape[2]])
tensor_seq_lengths = tf.reshape(tensor_seq_lengths,[tensor_seq_lengths.shape[0]*tensor_seq_lengths.shape[1],])
#seq2seq_out = get_encoded_rep()

In [264]:
seq2seq_out

<tf.Tensor 'transpose_4:0' shape=(2, 6, 50) dtype=float64>

In [5]:
%time
seq2seq_out.eval()
seq2seq_out.shape

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20 µs


TensorShape([Dimension(2), Dimension(6), Dimension(50)])

In [118]:
'''Inputs'''

#Input1 is of shape num_sentences(N)*max_sequence_length(L) and Input 2 is of shape N, (contains sequence lengths)
input_word_seq_tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[1,2,3,4,3,2]]))
max_seq_length = 6
seq_lengths = None #shape n

'''Create mask based on max seq length'''
np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
a = np.array(range(max_seq_length))
np_mask_matrix[np.arange(len(a)), a] = 0
#np_mask_matrix  = np_mask_matrix!=0
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")

padding = tf.constant([[0,0],[0,1]]) #remember it is of shape 1*dim_axis*2 (where the 2 are filled by paddings before and after)
'''Mask function '''
#below we first mask so N*max_sequence_length -> N*max_sequence_length-1
#then we pad a single value of 0 to each column to make result of same shape as max_seq_length 
#Padding makes it again into N*max_sequence_length

'''Resultant tensor: First we expand dimension and then in each loop concatenate a new row.
Resultant tensor after processing will have shape (max_sequence_length+1)*num_sentences*max_sequence_length
'''
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)

'''We store sequence lengths (useful for seq2seq) for each sequence in final_sequence-> if input was of shape num_sentences, after processing it will be of shape (max_seq_length+1)*num_sentences'''
'''Basically if it were [2,2,3] then it would be [[2,2,3],[1,1,2],[1,1,2],[1,1,2]'''
tensor_seq_lengths = tf.expand_dims(seq_lengths, 0)
seq_lengths_for_dropped = tf.expand_dims(seq_lengths-tf.ones(shape=seq_lengths.shape[0]))


for drop_index in range(max_seq_length):
    '''Create mask function-> note the drop index changes at each iteration'''
    f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
    '''Apply masking and padding'''
    temp = tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 
    resultant_tensor = tf.concat(resultant_tensor,temp,0)
    tensor_seq_lengths = tf.concat(tensor_seq_lengths, seq_lengths_for_dropped,0)



#convert returned tensor into the form (n*num_sen, max_len)
#give to seq2seq with seq_lens as 

In [124]:
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)
resultant_tensor, input_word_seq_tensor

(<tf.Tensor 'ExpandDims_2:0' shape=(1, 2, 6) dtype=int64>,
 <tf.Tensor 'Const_25:0' shape=(2, 6) dtype=int64>)

In [119]:
#below we first mask so N*max_sequence_length -> N*max_sequence_length-1
#then we pad a single value of 0 to each column to make result of same shape as max_seq_length 
#Padding makes it again into N*max_sequence_length

f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,0])#,padding,"CONSTANT") 

In [120]:
#Input tensor is of shape n
tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[1,2,3,4,3,2]]))

tf.pad(tf.map_fn(f, tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 


tf.concatenat

<tf.Tensor 'Pad_1:0' shape=(2, ?) dtype=int64>

In [106]:
tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[1,2,3,4,3,2]]))
#mask = np.array([True, False, True, False])
#tf.boolean_mask(tensor, mask)

ValueError: Shapes (2,) and (4,) are incompatible

In [95]:
tf_mask_matrix[:,3]

<tf.Tensor 'strided_slice_7:0' shape=(6,) dtype=bool>

In [ ]:
#convert input of batch_size*seq_length into 1*batch_size*seq_length
input_seq_tensor = tf.expand_dims(input_seq_tensor,0) 

In [ ]:
for i in range(seq_length):
    tf.boolean_mask(input_seq_tensor, tf_mask_matrix[:,i])

In [76]:
apply the np mask matrix 

array([[1, 2, 3, 0, 0, 0],
       [1, 2, 3, 0, 0, 0],
       [1, 2, 3, 0, 0, 0],
       [1, 2, 3, 0, 0, 0],
       [1, 2, 3, 0, 0, 0],
       [1, 2, 3, 0, 0, 0]])

In [77]:
tensor = x3
mask = np_mask_matrix
z = tf.boolean_mask(tensor, np_mask_matrix)

In [88]:
sess = tf.InteractiveSession()
z.eval()

array([2, 3, 0, 0, 0, 1, 3, 0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 3, 0, 0, 1, 2,
       3, 0, 0, 1, 2, 3, 0, 0])

In [61]:
x = tf.Variable(tf.truncated_normal(shape=(seq_length,seq_length,)))
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")

l = tf.boolean_mask(x,tf_mask_matrix)
l

<tf.Tensor 'boolean_mask_11/Gather:0' shape=(?,) dtype=float32>

In [46]:
x1 = np.array([[1,2,3,0,0,0],[3,4,5,6,0,0]])
#First output of [[[2,3,0,0,0],[4,5,6,0,0]] 1,3,0,0,0 1,2,0,0,0

In [64]:
x2 = np.repeat(x1, [6,6], axis =0).reshape(-1,6,6)
x2

array([[[1, 2, 3, 0, 0, 0],
        [1, 2, 3, 0, 0, 0],
        [1, 2, 3, 0, 0, 0],
        [1, 2, 3, 0, 0, 0],
        [1, 2, 3, 0, 0, 0],
        [1, 2, 3, 0, 0, 0]],

       [[3, 4, 5, 6, 0, 0],
        [3, 4, 5, 6, 0, 0],
        [3, 4, 5, 6, 0, 0],
        [3, 4, 5, 6, 0, 0],
        [3, 4, 5, 6, 0, 0],
        [3, 4, 5, 6, 0, 0]]])

In [72]:
np_mask_matrix

array([[False,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True],
       [ True,  True,  True, False,  True,  True],
       [ True,  True,  True,  True, False,  True],
       [ True,  True,  True,  True,  True, False]])

In [19]:
x1 = x1.reshape(1,2,6)

In [38]:
x1[0,:,[0,1,2]]

array([[1, 3],
       [2, 4],
       [3, 5]])

In [31]:
mask_matrix = [[1,1,0]]

In [35]:
seq_length = 6
np_mask_matrix = np.ones((seq_length,seq_length)) #num drops=n
a = np.array(range(seq_length))
#b = np.zeros((3, 4))
np_mask_matrix[np.arange(len(a)), a] = 0
np_mask_matrix

In [71]:
np_mask_matrix = np_mask_matrix!=0

In [ ]:
#Now given the input seq as a 3d tensor or 2d tensor, we apply mask in either step or we can form a repeat vector and apply the mask to the resultant matrix

In [6]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
y.eval().shape
y[]

<tf.Tensor 'strided_slice:0' shape=(10, 100) dtype=float32>

In [6]:
o = tf.ones(shape=(100,))
1 1 1 1 1 1 <- Num elements based on seq length
0 1 1 1 1 1 
1 0 1 1 1 1
1 1 0 1 1 1 
1 1 1 0 1 1 


In [7]:
"""Inputs:
Seq_len
Seq indices

Procedure:
Word_dropped_matrix = []
For i in seq_len:
    1) Make mask for each step of len = seq_len
    2) Apply mask to input seq row (entire row- each column) through above and map_fn in TF to produce the resultant
    3) word_dropped_matrix[i] = Batchwise
Resultant shape is seq_len*batch
"""

TypeError: 'Tensor' object does not support item assignment